# Welcome to explore...we're right in the thick of it 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

import prepare as p
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from scipy.stats import f_oneway



In [2]:
df = p.anna_prep()

In [3]:
df.dtypes

crash_date                   datetime64[ns]
crash_id                              int64
crash_latitude                      float64
crash_longitude                     float64
crash_occupant_count                  int64
crash_vehicle_count                   int64
damage_airbag                         int64
damage_burned                         int64
damage_concentrated                   int64
damage_distributed                    int64
damage_rollover                       int64
damage_zone                           int64
dl_cdl                                int64
dl_class_a                            int64
dl_class_b                            int64
dl_class_m                            int64
dl_state                             object
dl_unlicensed                         int64
driver_age                            int64
driver_male                           int64
driver_race                          object
factors_road                         object
factors_spd_lmt_mph             

In [4]:
train, test = train_test_split(df, test_size=0.2, random_state=19, stratify=df.injury_class)

In [5]:
train.shape, test.shape

((14352, 44), (3589, 44))

In [6]:
def split_X_y(train, test, target):
    '''
    Splits train, and test into a dataframe with independent variables
    and a series with the dependent, or target variable. 
    The function returns 2 dataframes and 2 series:
    X_train (df) & y_train (series), X_test & y_test. 
    '''

        
    # split train into X (dataframe, drop target) & y (series, keep target only)
    X_train = train.drop(columns=[target])
    y_train = train[target]
    
    # split test into X (dataframe, drop target) & y (series, keep target only)
    X_test = test.drop(columns=[target])
    y_test = test[target]
    
    return X_train, y_train, X_test, y_test

In [7]:
X_train, y_train, X_test, y_test = split_X_y(train, test, 'injury_class')

In [8]:
X_train

,crash_date,crash_id,crash_latitude,crash_longitude,crash_occupant_count,crash_vehicle_count,damage_airbag,damage_burned,damage_concentrated,damage_distributed,...,vehicle_color,vehicle_id,vehicle_make,vehicle_occupant_count,vehicle_type,vehicle_year,crash_time,crash_hour,crash_day_of_week,accident_contained_injury
6117,2021-04-12 15:57:00,18197524,29.608215,-98.508284,2,2,0,0,0,0,...,silver,3GCR*********0816,chevrolet,1,truck,2010,15:57:00,15,Monday,0
3463,2021-02-27 14:40:00,18126720,29.381912,-98.700943,2,1,0,0,0,0,...,red,1FM5*********6392,ford,2,mpv,2016,14:40:00,14,Saturday,0
794,2021-03-13 03:50:00,18152011,29.544563,-98.410983,2,2,0,0,1,0,...,black,1GKS*********7321,gmc,1,mpv,2015,03:50:00,3,Saturday,0
12646,2021-03-18 16:44:00,18159154,29.572367,-98.234738,4,2,0,0,0,1,...,gray,JTEB*********8464,toyota,3,mpv,2016,16:44:00,16,Thursday,0
12998,2021-03-17 12:21:00,18155591,29.435255,-98.479602,3,2,0,0,0,1,...,silver,WDZP*********6264,mercedes benz,2,bus,2012,12:21:00,12,Wednesday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2021-02-17 06:21:00,18127307,29.586975,-98.627732,2,2,0,0,0,0,...,gray,1FAF*********7104,ford,1,car,2004,06:21:00,6,Wednesday,0
7478,2021-04-07 14:54:00,18189241,29.367165,-98.516802,3,2,0,0,0,1,...,white,3FA6*********3077,ford,1,car,2019,14:54:00,14,Wednesday,1
15737,2021-05-03 11:45:00,18234414,29.474266,-98.264176,2,2,0,0,0,0,...,white,1HTM*********5339,other,1,non-passenger,2002,11:45:00,11,Monday,0
9503,2021-03-30 11:29:00,18191671,29.495961,-98.325525,2,2,1,0,1,0,...,white,1G1B*********8293,chevrolet,1,car,2018,11:29:00,11,Tuesday,0


In [9]:
def scale_data(train, test, scale_type = None, to_scale = None):
    '''
    returns scaled data of specified type into data frame, will
    '''
    train_copy = train.copy()
    test_copy = test.copy()
    
    if to_scale == None:
        return train_copy, test_copy
    
    else:
        X_train = train_copy[to_scale]
        X_test = test_copy[to_scale]
        
        
        min_max_scaler = MinMaxScaler()
        robust_scaler = RobustScaler()
        standard_scaler = StandardScaler()
        
        min_max_scaler.fit(X_train)
        robust_scaler.fit(X_train)
        standard_scaler.fit(X_train)
    
        mmX_train_scaled = min_max_scaler.transform(X_train)
        rX_train_scaled = robust_scaler.transform(X_train)
        sX_train_scaled = standard_scaler.transform(X_train)
    
    
        mmX_test_scaled = min_max_scaler.transform(X_test)
        rX_test_scaled = robust_scaler.transform(X_test)
        sX_test_scaled = standard_scaler.transform(X_test)
    
    
        mmX_train_scaled = pd.DataFrame(mmX_train_scaled, columns=X_train.columns)
        mmX_test_scaled = pd.DataFrame(mmX_test_scaled, columns=X_test.columns)

        rX_train_scaled = pd.DataFrame(rX_train_scaled, columns=X_train.columns)
        rX_test_scaled = pd.DataFrame(rX_test_scaled, columns=X_test.columns)


        sX_train_scaled = pd.DataFrame(sX_train_scaled, columns=X_train.columns)
        sX_test_scaled = pd.DataFrame(sX_test_scaled, columns=X_test.columns)
    
    
    if scale_type == 'MinMax':
        for i in mmX_train_scaled:
            train_copy[i] = mmX_train_scaled[i].values
            test_copy[i] = mmX_test_scaled[i].values
    elif scale_type == 'Robust':
        for i in rX_train_scaled:
            train_copy[i] = rX_train_scaled[i].values
            test_copy[i] = rX_test_scaled[i].values
    elif scale_type == 'Standard':
          for i in sX_train_scaled:
            train_copy[i] = sX_train_scaled[i].values
            test_copy[i] = sX_test_scaled[i].values
    return train_copy, test_copy
 

In [10]:
train_scaled, test_scaled = scale_data(train, test, scale_type = 'Standard', to_scale = ['crash_longitude','crash_latitude'])

In [11]:
train_scaled

,crash_date,crash_id,crash_latitude,crash_longitude,crash_occupant_count,crash_vehicle_count,damage_airbag,damage_burned,damage_concentrated,damage_distributed,...,vehicle_color,vehicle_id,vehicle_make,vehicle_occupant_count,vehicle_type,vehicle_year,crash_time,crash_hour,crash_day_of_week,accident_contained_injury
6117,2021-04-12 15:57:00,18197524,1.775196,-0.004821,2,2,0,0,0,0,...,silver,3GCR*********0816,chevrolet,1,truck,2010,15:57:00,15,Monday,0
3463,2021-02-27 14:40:00,18126720,-1.086878,-1.812980,2,1,0,0,0,0,...,red,1FM5*********6392,ford,2,mpv,2016,14:40:00,14,Saturday,0
794,2021-03-13 03:50:00,18152011,0.970189,0.908371,2,2,0,0,1,0,...,black,1GKS*********7321,gmc,1,mpv,2015,03:50:00,3,Saturday,0
12646,2021-03-18 16:44:00,18159154,1.321827,2.562488,4,2,0,0,0,1,...,gray,JTEB*********8464,toyota,3,mpv,2016,16:44:00,16,Thursday,0
12998,2021-03-17 12:21:00,18155591,-0.412247,0.264363,3,2,0,0,0,1,...,silver,WDZP*********6264,mercedes benz,2,bus,2012,12:21:00,12,Wednesday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2021-02-17 06:21:00,18127307,1.506571,-1.125877,2,2,0,0,0,0,...,gray,1FAF*********7104,ford,1,car,2004,06:21:00,6,Wednesday,0
7478,2021-04-07 14:54:00,18189241,-1.273388,-0.084769,3,2,0,0,0,1,...,white,3FA6*********3077,ford,1,car,2019,14:54:00,14,Wednesday,1
15737,2021-05-03 11:45:00,18234414,0.081125,2.286204,2,2,0,0,0,0,...,white,1HTM*********5339,other,1,non-passenger,2002,11:45:00,11,Monday,0
9503,2021-03-30 11:29:00,18191671,0.355513,1.710422,2,2,1,0,1,0,...,white,1G1B*********8293,chevrolet,1,car,2018,11:29:00,11,Tuesday,0


In [12]:
X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled = split_X_y(train_scaled, test_scaled, 'injury_class')

In [13]:
X_train_scaled

,crash_date,crash_id,crash_latitude,crash_longitude,crash_occupant_count,crash_vehicle_count,damage_airbag,damage_burned,damage_concentrated,damage_distributed,...,vehicle_color,vehicle_id,vehicle_make,vehicle_occupant_count,vehicle_type,vehicle_year,crash_time,crash_hour,crash_day_of_week,accident_contained_injury
6117,2021-04-12 15:57:00,18197524,1.775196,-0.004821,2,2,0,0,0,0,...,silver,3GCR*********0816,chevrolet,1,truck,2010,15:57:00,15,Monday,0
3463,2021-02-27 14:40:00,18126720,-1.086878,-1.812980,2,1,0,0,0,0,...,red,1FM5*********6392,ford,2,mpv,2016,14:40:00,14,Saturday,0
794,2021-03-13 03:50:00,18152011,0.970189,0.908371,2,2,0,0,1,0,...,black,1GKS*********7321,gmc,1,mpv,2015,03:50:00,3,Saturday,0
12646,2021-03-18 16:44:00,18159154,1.321827,2.562488,4,2,0,0,0,1,...,gray,JTEB*********8464,toyota,3,mpv,2016,16:44:00,16,Thursday,0
12998,2021-03-17 12:21:00,18155591,-0.412247,0.264363,3,2,0,0,0,1,...,silver,WDZP*********6264,mercedes benz,2,bus,2012,12:21:00,12,Wednesday,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5318,2021-02-17 06:21:00,18127307,1.506571,-1.125877,2,2,0,0,0,0,...,gray,1FAF*********7104,ford,1,car,2004,06:21:00,6,Wednesday,0
7478,2021-04-07 14:54:00,18189241,-1.273388,-0.084769,3,2,0,0,0,1,...,white,3FA6*********3077,ford,1,car,2019,14:54:00,14,Wednesday,1
15737,2021-05-03 11:45:00,18234414,0.081125,2.286204,2,2,0,0,0,0,...,white,1HTM*********5339,other,1,non-passenger,2002,11:45:00,11,Monday,0
9503,2021-03-30 11:29:00,18191671,0.355513,1.710422,2,2,1,0,1,0,...,white,1G1B*********8293,chevrolet,1,car,2018,11:29:00,11,Tuesday,0


### Does Hour of Day Affect Injuries?

$H$0: There is no difference in means for reported injuries by hour

$H$A: There is a difference in means for reported injuries by hour

In [14]:
twelve_am = train[train.crash_hour == 0].injury_class
one_am = train[train.crash_hour == 1].injury_class
two_am = train[train.crash_hour == 2].injury_class
three_am = train[train.crash_hour == 3].injury_class
four_am = train[train.crash_hour == 4].injury_class
five_am = train[train.crash_hour == 5].injury_class

six_am = train[train.crash_hour == 6].injury_class
seven_am = train[train.crash_hour == 7].injury_class
eight_am = train[train.crash_hour == 8].injury_class
nine_am = train[train.crash_hour == 9].injury_class
ten_am = train[train.crash_hour == 10].injury_class
eleven_am = train[train.crash_hour == 11].injury_class

twelve_pm = train[train.crash_hour == 12].injury_class
one_pm = train[train.crash_hour == 13].injury_class
two_pm = train[train.crash_hour == 14].injury_class
three_pm = train[train.crash_hour == 15].injury_class
four_pm = train[train.crash_hour == 16].injury_class
five_pm = train[train.crash_hour == 17].injury_class

six_pm = train[train.crash_hour == 18].injury_class
seven_pm = train[train.crash_hour == 19].injury_class
eight_pm = train[train.crash_hour == 20].injury_class
nine_pm = train[train.crash_hour == 21].injury_class
ten_pm = train[train.crash_hour == 22].injury_class
eleven_pm = train[train.crash_hour == 23].injury_class

In [15]:
f, p = f_oneway(twelve_am,one_am,two_am,three_am, four_am, five_am)
f,p

(1.1776466793920446, 0.31779029263218644)

In [16]:
f, p = f_oneway(six_am,seven_am,eight_am,nine_am, ten_am, eleven_am)
f,p

(1.1108831658474299, 0.35228950618519345)

In [17]:
f, p = f_oneway(twelve_pm,one_pm,two_pm,three_pm, four_pm, five_pm)
f,p

(0.9023615843415576, 0.47838530883249164)

In [18]:
f, p = f_oneway(six_pm,seven_pm,eight_pm,nine_pm, ten_pm, eleven_pm)
f,p

(1.390908114780306, 0.22428493763732993)

In [19]:
f, p = f_oneway(twelve_am,one_am,two_am,three_am, four_am, five_am, six_am,seven_am,eight_am,nine_am, ten_am, eleven_am,
                twelve_pm,one_pm,two_pm,three_pm, four_pm, five_pm, six_pm,seven_pm,eight_pm,nine_pm, ten_pm, eleven_pm)
                
f,p

(1.2069747995646756, 0.22523203179249)

 - since the p value of our anova test is .225 (which is greater than 0.05), we fail to reject the null hypothesis
 - there is not enough evidence to suggest a difference in injuries reported by hour

<br>
<br>

### Does Day of Week Affect Injuries?


$H$0: There is no difference in means for reported injuries by day of the week

$H$A: There is a difference in means for reported injuries by day of the week

In [22]:
monday = train[train.crash_day_of_week == 'Monday'].injury_class
tuesday = train[train.crash_day_of_week == 'Tuesday'].injury_class
wednesday = train[train.crash_day_of_week == 'Wednesday'].injury_class
thursday = train[train.crash_day_of_week == 'Thursday'].injury_class
friday = train[train.crash_day_of_week == 'Friday'].injury_class
saturday = train[train.crash_day_of_week == 'Saturday'].injury_class
sunday= train[train.crash_day_of_week == 'Sunday'].injury_class

In [23]:
f, p = f_oneway(monday, tuesday, wednesday, thursday, friday, saturday, sunday)
f,p

(1.5402019812253784, 0.16054873073006662)

 - since the p value of our anova test is .16 (which is greater than 0.05), we fail to reject the null hypothesis
 - there is not enough evidence to suggest a difference in injuries reported by day of the week

<br>
<br>